In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

!pip install ultralytics opencv-python-headless

import os
import cv2
import numpy as np

import torch
from google.colab import files

from ultralytics import YOLO

import tensorflow as tf

Mounted at /content/gdrive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 793.7/793.7 kB 4.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nv

In [2]:
playerDetectionModel = YOLO('/content/gdrive/MyDrive/Yolo v9c 100 Epoch train/weights/best.pt')

In [9]:
shotDetectionModel_path = '/content/gdrive/MyDrive/CroppedDetection/lrcn-tuned-model-last.keras'
shotDetectionModel = tf.keras.models.load_model(shotDetectionModel_path)

In [12]:
uploaded = files.upload()
video_path = ''

for file_name in uploaded.keys():
    video_path = 'demo.' + file_name.split('.')[-1]
    os.rename(file_name, video_path)

Saving 16.mp4 to 16.mp4


In [13]:
cap = cv2.VideoCapture(video_path)

shot_path = 'shot.' + video_path.split('.')[-1]

output_size = (256, 256)
out = cv2.VideoWriter(shot_path, cv2.VideoWriter_fourcc('M','J','P','G'), 30, output_size)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Perform detection
    results = playerDetectionModel(frame)

    # Flag to check if "Batsman" is detected
    batsman_detected = False

    for result in results:
        boxes = result.boxes
        for box in boxes:
            x1, y1, x2, y2 = box.xyxy[0].cpu().numpy().astype(int)
            cls = int(box.cls[0].cpu().numpy())
            label = playerDetectionModel.names[cls]

            if label == 'Batsman':  # Ensure the class name matches
                # Crop the frame to the bounding box
                cropped_frame = frame[y1:y2, x1:x2]

                # Resize the cropped frame to 256x256
                resized_frame = cv2.resize(cropped_frame, output_size)

                # Write the resized frame into the output file
                out.write(resized_frame)

                # Set the flag
                batsman_detected = True
                break

        if batsman_detected:
            break
        else:
          continue

    # Optionally display the frame (comment out if not needed)
    # cv2.imshow('Frame', resized_frame)
    # if cv2.waitKey(1) & 0xFF == ord('q'):
    #     break

# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()


0: 384x640 1 Batsman, 1 Bowler, 2 Fielders, 1 Keeper, 1 Non-strike-Batsman, 2653.0ms
Speed: 11.4ms preprocess, 2653.0ms inference, 2.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Batsman, 1 Bowler, 2 Fielders, 1 Keeper, 1 Non-strike-Batsman, 1892.0ms
Speed: 3.2ms preprocess, 1892.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Batsman, 1 Bowler, 2 Fielders, 1 Keeper, 1 Non-strike-Batsman, 1688.1ms
Speed: 2.7ms preprocess, 1688.1ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Batsman, 1 Bowler, 2 Fielders, 1 Keeper, 1 Non-strike-Batsman, 1695.0ms
Speed: 4.1ms preprocess, 1695.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Batsman, 2 Fielders, 1 Keeper, 1 Non-strike-Batsman, 1394.6ms
Speed: 2.2ms preprocess, 1394.6ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Batsman, 2 Fielders, 1 Keeper, 1 Non-strike-Batsman, 1856.0ms
Speed: 2

In [14]:
def extract_frames(shot_path, frame_count=10):
    cap = cv2.VideoCapture(shot_path)
    frames = []
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    interval = total_frames // frame_count

    for i in range(frame_count):
        cap.set(cv2.CAP_PROP_POS_FRAMES, i * interval)
        ret, frame = cap.read()
        if ret:
            frame = cv2.resize(frame, (64, 64))
            frames.append(frame)
        else:
            break
    cap.release()
    return np.array(frames)

In [15]:
new_frames = extract_frames(shot_path)
new_frames = np.expand_dims(new_frames, axis=0)

prediction = shotDetectionModel.predict(new_frames)
predicted_class = np.argmax(prediction, axis=1)
print(prediction)

shot = {0: 'Defensive Shot', 1: 'Pull Shot', 2: 'Drive Shot', 3: 'Flick Shot'}
print(shot[predicted_class[0]])

1/1 [==============================] - 1s 870ms/step
[[    0.99017  4.7525e-05  0.00074293   0.0090406]]
Defensive Shot
